<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 20px; height: 55px">

# Project 3: Linear Regression and Train/Test Split

---

# Introduction

We've discussed overfitting in the context of bias and variance, and we've touched on some techniques that are used to avoid overfitting (but haven't practiced them yet). In this project, we'll practice a fundamental method for avoiding overfitting that is commonly referred to as: _train/test split validation_. 

This is similar to something called "cross-validation" — in fact, it is a type of cross-validation — in that we will be splitting the data into two subsets:
* A subset on which to train our model.
* A subset on which to test our model's predictions.

This serves two useful purposes:
* We prevent overfitting by not using all of the data.
* We have some remaining data we can use to evaluate our model.

While this may seem like a relatively simple idea, **there are some caveats** to putting it into practice. For example, if you are not careful, it is easy to take a non-random split. Suppose we have salary data on technical professionals that is composed of 80 percent data from California, 20 percent data from elsewhere, and is sorted by state. If we split our data into 80 percent training data and 20 percent testing data, we might inadvertantly select all the California data to train and all the non-California data to test. In this case, we've still overfit on our data set because we did not sufficiently randomize the data.

Note: In a situation like this, you could use a technique called _k-fold cross-validation_, which is cross-validation applied to more than two subsets. In particular, in k-fold cross-validation, you'd partition your data into $k$ subsets and train on $k-1$ one of them, holding the last slice for testing. You would do this for each of the possible $k-1$ subsets.

# Project Requirements

The goal of this project will be to create a test-training split to compare multiple models on the same Boston housing data set. For example, you might choose to compare different linear models on the same data.

In this project, you'll fit two to three different models on the Boston housing data. For example, you could pick two different subsets of variables, one or more polynomial models, or any other model you'd like. 

### Here's What We Will Be Doing:

* Work with Boston housing data to predict the value of a home
* Create a test-train split of the data.
* Train your model on the training data.
* Evaluate your model on the test data.
* Repeat with other variables.
* Rank the models by how well they score on the testing data set. 
* Don't forget to interpret your findings! 



A k-fold cross-validation creates a hold portion of your data set for each iteration of training and validating:

![](http://i.imgur.com/0PFrPXJ.png)

## Linear Regression

As a real estate developer, you are being asked to model the median home price of various houses in the city of Boston using data from the US census. Your goal is to predict a continuous, numeric output (price) based on a combination of discrete features that you choose.

In [1]:
import matplotlib.pyplot as plt
%matplotlib inline

In [5]:
import pandas as pd
import numpy as np
from sklearn.datasets import load_boston
boston = load_boston()

all_data = pd.DataFrame(boston.data,
                 columns=boston.feature_names)

all_data["MEDV"] = boston.target
#print(boston['DESCR'])

### REQUIRED: Clean Up Data and Perform Exporatory Data Analysis

In [6]:
all_data["MEDV"].mean()

22.532806324110698

Boston data is from scikit-learn, so it _ought_ to be pretty clean, but you should always perform exploratory data analysis.

In [7]:
# Exploratory data analysis.

# Include: total nulls, index, data types, shape, summary statistics, and the number of unique values for each column

In [8]:
all_data.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2


In [9]:
all_data.tail()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
501,0.06263,0.0,11.93,0.0,0.573,6.593,69.1,2.4786,1.0,273.0,21.0,391.99,9.67,22.4
502,0.04527,0.0,11.93,0.0,0.573,6.120,76.7,2.2875,1.0,273.0,21.0,396.90,9.08,20.6
503,0.06076,0.0,11.93,0.0,0.573,6.976,91.0,2.1675,1.0,273.0,21.0,396.90,5.64,23.9
504,0.10959,0.0,11.93,0.0,0.573,6.794,89.3,2.3889,1.0,273.0,21.0,393.45,6.48,22.0
505,0.04741,0.0,11.93,0.0,0.573,6.030,80.8,2.5050,1.0,273.0,21.0,396.90,7.88,11.9


In [10]:
# total nulls

all_data.isna().sum()

CRIM       0
ZN         0
INDUS      0
CHAS       0
NOX        0
RM         0
AGE        0
DIS        0
RAD        0
TAX        0
PTRATIO    0
B          0
LSTAT      0
MEDV       0
dtype: int64

In [11]:
# index

all_data.index

RangeIndex(start=0, stop=506, step=1)

In [12]:
# data types

all_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 506 entries, 0 to 505
Data columns (total 14 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   CRIM     506 non-null    float64
 1   ZN       506 non-null    float64
 2   INDUS    506 non-null    float64
 3   CHAS     506 non-null    float64
 4   NOX      506 non-null    float64
 5   RM       506 non-null    float64
 6   AGE      506 non-null    float64
 7   DIS      506 non-null    float64
 8   RAD      506 non-null    float64
 9   TAX      506 non-null    float64
 10  PTRATIO  506 non-null    float64
 11  B        506 non-null    float64
 12  LSTAT    506 non-null    float64
 13  MEDV     506 non-null    float64
dtypes: float64(14)
memory usage: 55.5 KB


In [13]:
# shape

all_data.shape

(506, 14)

In [14]:
# summary statistics

all_data.describe()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
count,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000
mean,3.613524,11.363636,11.136779,0.069170,0.554695,6.284634,68.574901,3.795043,9.549407,408.237154,18.455534,356.674032,12.653063,22.532806
std,8.601545,23.322453,6.860353,0.253994,0.115878,0.702617,28.148861,2.105710,8.707259,168.537116,2.164946,91.294864,7.141062,9.197104
min,0.006320,0.000000,0.460000,0.000000,0.385000,3.561000,2.900000,1.129600,1.000000,187.000000,12.600000,0.320000,1.730000,5.000000
25%,0.082045,0.000000,5.190000,0.000000,0.449000,5.885500,45.025000,2.100175,4.000000,279.000000,17.400000,375.377500,6.950000,17.025000
50%,0.256510,0.000000,9.690000,0.000000,0.538000,6.208500,77.500000,3.207450,5.000000,330.000000,19.050000,391.440000,11.360000,21.200000
75%,3.677083,12.500000,18.100000,0.000000,0.624000,6.623500,94.075000,5.188425,24.000000,666.000000,20.200000,396.225000,16.955000,25.000000
max,88.976200,100.000000,27.740000,1.000000,0.871000,8.780000,100.000000,12.126500,24.000000,711.000000,22.000000,396.900000,37.970000,50.000000


In [15]:
# unique values

list(all_data.columns)

uniqueValues = all_data.nunique()

print(uniqueValues)

CRIM       504
ZN          26
INDUS       76
CHAS         2
NOX         81
RM         446
AGE        356
DIS        412
RAD          9
TAX         66
PTRATIO     46
B          357
LSTAT      455
MEDV       229
dtype: int64


## Using `scikit-learn` Linear Regression

### REQUIRED: Pick 3-4 predictors (i.e. CRIM, ZN, etc...) that you will use to predict your target variable, MEDV.
Score and plot your predictions. What do these results tell us?

In [17]:
# # predictors of choice: DIS, PTRATIO, B, LSTAT 

# clean_data = all_data[["PTRATIO", "B", "LSTAT"]]

# X = clean_data
# y = pd.DataFrame(boston.target,
#                  columns=['MEDV'])

# predictors of choice: DIS, PTRATIO, B, LSTAT 

X = all_data[["PTRATIO", "B", "LSTAT"]]

y = all_data["MEDV"]

## REQUIRED: Try 70/30 and 90/10 train/test splits: 70% of the data for training - 30% for testing, then 90% for training - 10% for testing.
Score and plot both of these approaches. How do your metrics change? What does this tell us about the size of training/testing splits? Include a written response.

## BONUS: Try the k-fold cross-validation technique described above, varying the number of folds from 5 to 10
What seems optimal? How do your scores change? What is the variance like? Try different folds to get a sense of how this impacts your score. What are the tradeoffs associated with choosing the number of folds? Include a written response.

## 70/30 train/test split - Cross Val = 5

In [18]:
from sklearn.model_selection import train_test_split
# allocate balance of target test size (30%) as training size (70%) at random to training data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42)

In [19]:
from sklearn import linear_model
reg = linear_model.LinearRegression()

In [20]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(reg, X_train, y_train, cv=5, scoring="r2")

print(scores)

print("R2 = " + str(scores.mean()))

[0.61357721 0.68609434 0.57647702 0.71126376 0.44980928]
R2 = 0.6074443221543726


In [21]:
reg.fit(X_train, y_train)

LinearRegression()

In [22]:
preds = reg.predict(X_test)

preds

array([27.88400554, 31.44441021, 16.66317658, 28.06696968, 17.18341533,
       23.06870689, 18.22459833, 10.42380385, 20.93602433, 20.77555209,
       20.70434007, 22.55378822, -1.73251159, 22.86728567, 22.46826067,
       21.77106732, 18.02448798,  5.95625789, 35.22634269, 16.56585528,
       27.85138161, 28.39440255, 11.69819344, 22.74025096, 17.94535492,
       16.18559263, 25.64835827, 12.80783099, 18.9207288 , 20.96721924,
       23.18740905, 26.3000248 , 20.47419843, 23.04734495, 13.35692229,
       23.11999448, 30.784983  , 23.50753938, 20.99561882, 26.26832732,
       15.10053784, 27.27665075, 34.957554  , 22.12179468, 26.79618492,
       17.39512941, 15.22812033, 27.00671328, 18.09919246, 32.63502489,
       25.83097643, 33.33804438, 15.23976962, 24.54135098, 36.49525984,
       22.78805276, 17.83903763, 29.94224178, 25.43067749, 17.37297389,
       24.26797379, 30.05669406, 31.32237753, 20.19254704, 28.86176064,
       14.10134708, 17.36623588, 27.32583912, 30.04530196, 12.58

In [23]:
evaluate = pd.DataFrame({
    "actual" : y_test
    , "predicted" : preds
}
)
evaluate["error"] = evaluate["actual"] - evaluate["predicted"]
evaluate.head()

,actual,predicted,error
173,23.6,27.884006,-4.284006
274,32.4,31.444410,0.955590
491,13.6,16.663177,-3.063177
72,22.8,28.066970,-5.266970
452,16.1,17.183415,-1.083415


## 90/10 train/test split - Cross Val = 7

In [27]:
from sklearn.model_selection import train_test_split

# allocate balance of target test size (30%) as training size (70%) at random to training data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.10, random_state=42)

In [29]:
from sklearn.model_selection import cross_val_score

scores = cross_val_score(reg, X_train, y_train, cv=7, scoring="r2")

print(scores)

print("R2 = " + str(scores.mean()))

[0.5365892  0.59176771 0.56148575 0.66261079 0.62220511 0.70119157
 0.49353038]
R2 = 0.5956257863086554


In [30]:
reg.fit(X_train, y_train)

LinearRegression()

In [31]:
preds = reg.predict(X_test)

preds

array([27.76311908, 31.07581173, 16.65633283, 27.68315224, 17.1513765 ,
       22.96052623, 18.32386375, 10.56415322, 20.7380374 , 20.68261539,
       20.89544068, 22.80216112, -0.62909929, 22.75180963, 22.2729744 ,
       21.56410754, 18.07545695,  6.39819308, 35.02762253, 16.54467299,
       27.5293188 , 28.15053463, 11.84053452, 22.39223055, 17.96153819,
       16.23399593, 25.4073724 , 13.4160877 , 18.96833654, 20.86644428,
       23.25688379, 25.94100316, 20.29318381, 22.78074231, 13.54696116,
       23.54751987, 30.42823366, 23.20247579, 20.97291859, 25.97481739,
       15.23968176, 26.99817201, 34.74419209, 22.00392371, 26.6033506 ,
       17.34304522, 15.22781135, 26.67500902, 18.0153041 , 32.44866022,
       25.9329046 ])

In [32]:
evaluate = pd.DataFrame({
    "actual" : y_test
    , "predicted" : preds
}
)
evaluate["error"] = evaluate["actual"] - evaluate["predicted"]
evaluate.head()

,actual,predicted,error
173,23.6,27.763119,-4.163119
274,32.4,31.075812,1.324188
491,13.6,16.656333,-3.056333
72,22.8,27.683152,-4.883152
452,16.1,17.151377,-1.051377


## Assesment

After selecting "PTRATIO", "B", "LSTAT" as the features and comparing the model's performance (measured by based on R-squared) using 5-fold cross tabulation and 7-fold cross tabulation. The finding were:

1. 7-fold cross tabulation produced a lower R-squared score than 5-fold cross tabulation.
2. The benefit of using "PTRATIO", "B", "LSTAT" as predictive features seems reasonable. However further investigation is required namely: a) the cost-benefit of further optimisation (e.g. 10-fold cross tabulation); and, b) a comparison against another feature set combinate to determine value.

## BONUS
Practice the same workflow you just worked through above, but this time use Statsmodels instead of Sklearn for linear regression. See example formulas below.

### Example: Statsmodels Formulas

Adjust the formula using your chosen metrics from the housing data. Remember, your workflow here is the same, but the syntax is a little different. Describe your results. Are they comparable to scikit-learn's regression models?

In [ ]:
# First, format your data in a DataFrame

df = pd.DataFrame(boston.data, columns=boston.feature_names)
df['MEDV'] = boston.target
df.head()

In [ ]:
# Set up your new statsmodel.formula handling model
import statsmodels.formula.api as smf

# You can easily swap these out to test multiple versions/different formulas
formulas = {
    "case1": "MEDV ~ RM + LSTAT + RAD + TAX + NOX + INDUS + CRIM + ZN - 1", # - 1 = remove intercept
    "case2": "MEDV ~ NOX + RM",
    "case3": "MEDV ~ RAD + TAX"
}

model = smf.ols(formula=formulas['case1'], data=df)
result = model.fit()

result.summary()

### BONUS:

Can you optimize your R2, selecting the best features? Provide your code and explain your process in a brief written response.

### BONUS:

Given a combination of predictors, can you identify another response variable that could be accurately predicted through the exploration of different predictors in this data set?

_Tip: Consider pairplots, coefficients, or Pearson scores._

In [ ]:
# Check out variable relations
import seaborn as sns

sns.pairplot(X);

In [ ]:
# Check out Pearson scores


## BONUS/DEMO: Up for an additional challenge? Try again, this time using the `patsy` library.

In [ ]:
import patsy

# Add response to the core DataFrame
df['MEDV'] = y

In [ ]:
from sklearn.model_selection import train_test_split  #Note: if you didn't import this earlier, you'll need to do so now

# Easily change your variable predictors without reslicing your DataFrame
y, X = patsy.dmatrices("MEDV ~ AGE + RM", data=df, return_type="dataframe")

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=.7)


In [ ]:
from sklearn import metrics
from sklearn.linear_model import LinearRegression

# Rerun your model, iteratively changing your variables and train_size from the previous cell

lm = LinearRegression()
model = lm.fit(X_train, y_train)

predictions = model.predict(X_test)
print("R^2 Score: {}".format(metrics.r2_score(y_test, predictions)))